### Import Libraries

In [1]:
import numpy as np
import pandas as pd 

from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential, Model,load_model
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalMaxPooling2D
from keras.layers import BatchNormalization, merge, Input, Activation, Dropout, Flatten, Dense 
from keras import backend as K
from keras.layers import LeakyReLU
from keras.applications import VGG16
from sklearn.metrics import accuracy_score
from keras import layers
from keras import optimizers

import matplotlib.pyplot as plt
import tensorflow as tf

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
import random
import os

Using TensorFlow backend.


In [2]:
input_shape = (128, 128, 3)

### Import VGG16

To Solve this classification problem, we use VGG16. A state of the art classification model which is pre trained for classifying such image instances. We use Weights given by imagenet. <br>
The model is predefined and has been optimized for these weights.

In [4]:
pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")

In [4]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

In [5]:
x = GlobalMaxPooling2D()(last_output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = layers.Dense(2, activation='sigmoid')(x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
model = Model(pre_trained_model.input, x)
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])
model.summary()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_______________________________________________

In [7]:
# model.load_weights('VGG_v1.h5')

### Loading Data and Test-Train split

In [8]:
filenames = os.listdir(r'C:\Users\pmven\Downloads\dogs-vs-cats\train\\')
test_filenames = os.listdir(r'C:\Users\pmven\Downloads\dogs-vs-cats\test\\')

categories = []

for i in filenames:
    category = i.split('.')[0]
    if category == 'dog':
        categories.append('dog')
    else:
        categories.append('cat')

df = pd.DataFrame({'filenames':filenames,'categories':categories})

In [9]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
test_df = pd.DataFrame({'filename': test_filenames})

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
total_test = test_df.shape[0]

We use Image generators to convert these images into Matrix format which can be easily read by the Neural Network. WE use a target size of 128 x 128 for the same.

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(dataframe = train_df,  directory = r'C:\Users\pmven\Downloads\dogs-vs-cats\train\\',  x_col='filenames', \
                                                    y_col='categories',  class_mode="categorical", target_size=(128,128), batch_size = 15)

validation_generator = train_datagen.flow_from_dataframe( dataframe = validate_df,  directory = r'C:\Users\pmven\Downloads\dogs-vs-cats\train\\',  x_col='filenames', \
                                                          y_col='categories', class_mode="categorical", target_size=(128,128), batch_size = 15)

test_generator = test_gen.flow_from_dataframe(test_df,  directory = r'C:\Users\pmven\Downloads\dogs-vs-cats\test\\', x_col='filename', y_col=None,
                                              class_mode=None,  target_size=(128,128), batch_size=15, shuffle=False)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Found 12500 validated image filenames.


### Model Training

In [11]:
history = model.fit_generator(train_generator, epochs=5, validation_data=validation_generator, validation_steps=total_validate//16, steps_per_epoch=total_train//16)

Epoch 1/5
1250/1250 [==============================] - 168s 134ms/step - loss: 0.2719 - acc: 0.8737 - val_loss: 0.1393 - val_acc: 0.9463
Epoch 2/5
1250/1250 [==============================] - 153s 122ms/step - loss: 0.1137 - acc: 0.9561 - val_loss: 0.0974 - val_acc: 0.9615
Epoch 3/5
1250/1250 [==============================] - 154s 123ms/step - loss: 0.0839 - acc: 0.9667 - val_loss: 0.0809 - val_acc: 0.9679
Epoch 4/5
1250/1250 [==============================] - 163s 130ms/step - loss: 0.0648 - acc: 0.9757 - val_loss: 0.0849 - val_acc: 0.9649
Epoch 5/5
1250/1250 [==============================] - 163s 130ms/step - loss: 0.0456 - acc: 0.9834 - val_loss: 0.0859 - val_acc: 0.9654


After Multiple Trial and Errors, We finalize on 5 epochs for the model to reduce overfitting. 

In [18]:
model.save('VGG_model2.h5')

### Predictions

In [12]:
predict = model.predict_generator(test_generator, steps=np.ceil(total_test/15), verbose=1)

834/834 [==============================] - 86s 103ms/step


In [14]:
test_df['label'] = list(predict[:,1])

In [15]:
train_df.shape

(20000, 2)

In [16]:
test_df['id'] = test_df['filename'].str.replace('.jpg', '')
test_df['id'] = test_df['id'].astype(int)
test_df.sort_values(by = 'id')
test_df.head()

,filename,label,id
0,1.jpg,0.999992,1
1,10.jpg,0.000052,10
2,100.jpg,0.000625,100
3,1000.jpg,0.999948,1000
4,10000.jpg,0.913598,10000


In [17]:
test_df[['id', 'label']].to_csv('results_vgg_v3.csv', index = False)

### Results

![imge](results_vgg_v1.png)

These redults can be further improved by Stacking various such algorithms, to get the logloss score even higher.